In [1]:
import numpy as np
import pandas as pd
import quandl
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [8]:
quandl.ApiConfig.api_key = 'tAyfv1zpWnyhmDsp91yv'
spy_table = quandl.get('BCIW/_SPXT')
amzn_table = quandl.get('WIKI/AMZN')
ebay_table = quandl.get('WIKI/EBAY')
wal_table = quandl.get('WIKI/WMT')
aapl_table = quandl.get('WIKI/AAPL')

In [ ]:
spy = spy_table.loc['2016',['Close']]
amzn = amzn_table.loc['2016',['Close']]
ebay = ebay_table.loc['2016',['Close']]
wal = wal_table.loc['2016',['Close']]
aapl = aapl_table.loc['2016',['Close']]
spy_log = np.log(spy.Close).diff().dropna()
amzn_log = np.log(amzn.Close).diff().dropna()
ebay_log = np.log(ebay.Close).diff().dropna()
wal_log = np.log(wal.Close).diff().dropna()
aapl_log = np.log(aapl.Close).diff().dropna()
df = pd.concat([spy_log,amzn_log,ebay_log,wal_log,aapl_log],axis = 1).dropna()
df.columns = ['spy','amzn','ebay','wal','aapl']
df.tail()

In [ ]:
model = sm.ols(formula = 'amzn ~ spy+ebay+wal+aapl',data = df).fit()
print model.summary()

In [ ]:
simple = sm.ols(formula = 'amzn ~ spy',data = df).fit()
print simple.summary()

In [ ]:
fama_table = quandl.get('KFRENCH/FACTORS5_D')

In [ ]:
fama = fama_table['2016']
fama = fama.rename(columns = {'Mkt-RF':'MKT'})
fama = fama.apply(lambda x: x/100)
fama_df = pd.concat([fama,amzn_log],axis = 1)

In [ ]:
fama_model = sm.ols(formula = 'Close~MKT+SMB+HML+RMW+CMA',data = fama_df).fit()
print fama_model.summary()

In [ ]:
result = pd.DataFrame({'simple regression':simple.predict(),'fama_french':fama_model.predict(),'sample':df.amzn},index = df.index)
plt.figure(figsize = (15,7.5))
plt.plot(result['2016-7':'2016-9'].index,result.loc['2016-7':'2016-9','simple regression'])
plt.plot(result['2016-7':'2016-9'].index,result.loc['2016-7':'2016-9','fama_french'])
plt.plot(result['2016-7':'2016-9'].index,result.loc['2016-7':'2016-9','sample'])
plt.legend()
plt.show()

In [ ]:
plt.figure()
simple.resid.plot.density()
plt.show()

In [ ]:
print 'residual mean: ', np.mean(fama_model.resid)
print 'residual variance: ', np.var(fama_model.resid)

In [ ]:
plt.figure(figsize = (20,10))
plt.scatter(df.spy,simple.resid)
plt.axhline(0.05,color = 'r')
plt.axhline(-0.05,color = 'r')
plt.axhline(0,color = 'black')
plt.xlabel('x value')
plt.ylabel('residual')
plt.show()

In [ ]:
from statsmodels.stats import diagnostic as dia

In [ ]:
het = dia.het_breuschpagan(fama_model.resid,fama_df[['MKT','SMB','HML','RMW','CMA']][1:])
print 'p-value of Heteroskedasticity: ', het[-1]

In [3]:
dia.het_breuschpagan(simple.resid,pd.DataFrame(df.spy))

NameError: name 'dia' is not defined

In [4]:
simple.resid.shape

NameError: name 'simple' is not defined

In [5]:
df.spy.shape

NameError: name 'df' is not defined

In [6]:
# fama-french factor explore##
fama_table = quandl.get('KFRENCH/FACTORS5_M')
fama = fama_table['2008':'2016']

AuthenticationError: (Status 400) (Quandl Error QEAx01) We could not recognize your API key: _fgkxjSbt5389zGt4crC. Please check your API key and try again. You can find your API key under your account settings.

In [7]:
fama = fama.apply(lambda x: x/100)
fama.tail()

NameError: name 'fama' is not defined

In [98]:
amzn = amzn_table.loc['2008':'2016',['Close']]
amzn = amzn.resample('M').agg(lambda x: x[-1])
amzn_log = np.log(amzn.Close).diff().dropna()
amzn_log.tail()

Date
2016-08-31    0.013548
2016-09-30    0.084895
2016-10-31   -0.058389
2016-11-30   -0.050972
2016-12-31   -0.000933
Freq: M, Name: Close, dtype: float64

In [100]:
new_df = pd.concat([fama,amzn_log],axis = 1).dropna()
new_df = new_df.rename(columns = {'Mkt-RF':'MKT'})
new_df.tail()

,MKT,SMB,HML,RMW,CMA,RF,Close
Date,,,,,,,
2016-08-31,0.0050,0.0152,0.0318,-0.0124,-0.0056,0.0002,0.013548
2016-09-30,0.0025,0.0172,-0.0134,-0.0185,-0.0005,0.0002,0.084895
2016-10-31,-0.0202,-0.0397,0.0415,0.0136,0.0022,0.0002,-0.058389
2016-11-30,0.0486,0.0703,0.0844,-0.0068,0.0383,0.0001,-0.050972
2016-12-31,0.0182,0.0032,0.0352,0.0095,-0.0021,0.0003,-0.000933


In [128]:
m = sm.ols(formula = 'Close ~ MKT + SMB + HML',data = new_df).fit()
print m.summary()

                            OLS Regression Results                            
Dep. Variable:                  Close   R-squared:                       0.022
Model:                            OLS   Adj. R-squared:                  0.012
Method:                 Least Squares   F-statistic:                     2.333
Date:                Wed, 19 Jul 2017   Prob (F-statistic):              0.130
Time:                        15:40:51   Log-Likelihood:                 102.72
No. Observations:                 107   AIC:                            -201.4
Df Residuals:                     105   BIC:                            -196.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0198      0.009      2.184      0.0